In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib notebook

## Read in netCDF data

In [5]:
from scipy.io import netcdf_file

# https://resources.marine.copernicus.eu/product-download/SEALEVEL_EUR_PHY_L4_MY_008_068
#dataset = 'data10102019'
dataset = "01052014"
f = netcdf_file(f"{dataset}.nc")
f.variables

{'ugos': <scipy.io._netcdf.netcdf_variable at 0x7fc3ce3713d0>,
 'vgos': <scipy.io._netcdf.netcdf_variable at 0x7fc3ce2fe820>,
 'vgosa': <scipy.io._netcdf.netcdf_variable at 0x7fc3ce2fe850>,
 'crs': <scipy.io._netcdf.netcdf_variable at 0x7fc3ce2fe8e0>,
 'err_vgosa': <scipy.io._netcdf.netcdf_variable at 0x7fc3ce2fe970>,
 'latitude': <scipy.io._netcdf.netcdf_variable at 0x7fc3ce2fea30>,
 'sla': <scipy.io._netcdf.netcdf_variable at 0x7fc3ce2feac0>,
 'ugosa': <scipy.io._netcdf.netcdf_variable at 0x7fc3ce2feb50>,
 'flag_ice': <scipy.io._netcdf.netcdf_variable at 0x7fc3ce2feca0>,
 'adt': <scipy.io._netcdf.netcdf_variable at 0x7fc3ce2fed30>,
 'err_ugosa': <scipy.io._netcdf.netcdf_variable at 0x7fc3ce2fedf0>,
 'time': <scipy.io._netcdf.netcdf_variable at 0x7fc3ce2fee80>,
 'longitude': <scipy.io._netcdf.netcdf_variable at 0x7fc3ce2feee0>,
 'err_sla': <scipy.io._netcdf.netcdf_variable at 0x7fc3ce2fef70>}

In [6]:
lat = f.variables['latitude'].data
long = f.variables['longitude'].data
vel_x = f.variables['ugos'].data.squeeze()
print(vel_x.shape[0])
vel_y = f.variables['vgos'].data.squeeze()
print(vel_y.shape)
from common_functions import interpolate_missing_point

imputed_vel_x = interpolate_missing_point(vel_x, np.ma.masked_invalid(vel_x).mask)
imputed_vel_y = interpolate_missing_point(vel_y, np.ma.masked_invalid(vel_y).mask)

320
(320, 81)


In [43]:
from scipy.interpolate import RegularGridInterpolator
from scipy.integrate import solve_ivp
from scipy.spatial.distance import euclidean

# first build an interpolator over the x and y velocity
# for now dont use any proper coordinates just [0,m]*[0,n]
m,n = imputed_vel_x.shape
i = np.linspace(0, m, m)
j = np.linspace(0, n, n)
method = "linear"

interp_vel_x = RegularGridInterpolator((i, j), imputed_vel_x, method=method)
interp_vel_y = RegularGridInterpolator((i, j), imputed_vel_y, method=method)

def rhs(t, x):
        return [interp_vel_x(x).squeeze(), interp_vel_y(x).squeeze()]

def streamline(i,j, *, steps, step_size, dist_tol, winding_tol):
    #print(i,j, flush=True)
    winding_angle = 0
    theta_prev = 0
    closed = False

    start_i = i
    start_j = j

    # streamline path
    trajectory = [[i,j]]

    for k in range(steps):
        try:
            # get velocities
            v = rhs(0,[i,j])
        except ValueError:
            # stream line leaves domain
            break

        # integrate
        i += step_size*v[1]
        j += step_size*v[0]

        # get angle
        theta = np.arctan2(v[1], v[0])

        if k > 1:
            winding_angle += theta - theta_prev

        if k > 100 and (dist:=euclidean([i,j],[start_i,start_j])) < dist_tol:
            closed = True
            break


        theta_prev = theta
        trajectory.append([i,j])

    if (winding_angle % (2*np.pi) < winding_tol or winding_angle % (-2*np.pi) < winding_tol) and closed:
        #print(f"{dist=} {winding_angle=} {closed=}")
        return np.asarray(trajectory).T
    return None

def winding(dist_tol, winding_tol):
    from multiprocessing import Pool
    from functools import partial
    from itertools import product

    step = 5

    i_points = range(0, imputed_vel_x.shape[0], step)
    j_points = range(0, imputed_vel_x.shape[1], step)
    
    with Pool() as p:
        func = partial(streamline, steps=5000, step_size=0.1, dist_tol=dist_tol, winding_tol=winding_tol)

        res = p.starmap(func, product(i_points,j_points))

    eddies = []

    for result in res:
        if result is not None:
            eddies.append(np.mean(result, axis=-1))

    eddies = np.asarray(eddies)

    m,n = imputed_vel_x.shape
    i = np.linspace(0, m, m)
    j = np.linspace(0, n, n)
    method = "linear"

    latm, longm = np.meshgrid(lat, long, indexing='ij')

    interp_lat = RegularGridInterpolator((i, j), latm, method=method)
    interp_long = RegularGridInterpolator((i, j), longm, method=method)
    return np.column_stack([interp_lat(eddies), interp_long(eddies)])

In [44]:
from dataclasses import dataclass

from scipy.spatial import KDTree
from sklearn.metrics.pairwise import haversine_distances
import pandas as pd

real_data = pd.read_csv("validation_data/01052014.csv")
real_data = real_data[["Lat","Lon"]].to_numpy()
real_data = real_data[(real_data[:,1] > -30) & (real_data[:,1] < -20) & (real_data[:,0] > 20) & (real_data[:,0] < 60)]

@dataclass
class Method:
    name: str
    points: np.ndarray
    P: int = 0
    TP: int = 0
    FP: int = 0
        
    def compare(self, ground_truth: np.ndarray, dist_tol):
        '''dist_tol is in km'''
        self.P = ground_truth.shape[0]
        
        tree_detected = KDTree(self.points)
        dist, index = tree_detected.query(ground_truth[:,0:2])
        truth = np.deg2rad(ground_truth[:,0:2])
        detected = np.deg2rad(self.points[index,:])
        dist_mat = haversine_distances(truth, detected)
        dist = np.diagonal(dist_mat) * 6371000/1000  # multiply by Earth radius to get kilometers
        
        self.TP = (dist < dist_tol).sum()
        self.mean_error = np.mean(dist[dist < dist_tol])
        self.FN = self.P - self.TP
        
        tree_truth = KDTree(ground_truth[:,0:2])
        dist, index = tree_truth.query(self.points)
        detected = np.deg2rad(ground_truth[index,0:2])
        dist_mat = haversine_distances(truth, detected)
        dist = np.diagonal(dist_mat) * 6371000/1000  # multiply by Earth radius to get kilometers
                
        self.FP = (dist > dist_tol).sum()
    
    @property
    def TPR(self):
        return self.TP/self.P
    
    @property
    def FDR(self):
        return self.FP/(self.FP + self.TP)
    
    @property
    def F1(self):
        return self.TP/(self.TP + 0.5*(self.FP + self.FN))

In [45]:
res = np.empty((10,10))
for i, a in enumerate(np.linspace(0,1,5)):
    for j, b in enumerate(np.linspace(0,1,5)):
        print(i,j, flush=True)
        method = Method("",winding(a,b))
        try:
            method.compare(real_data, 50)
            res[i,j] = method.F1
        except IndexError:
            res[i,j] = 0

0 0
0 1
0 2
0 3
0 4
1 0
1 1
1 2
1 3
1 4
2 0
2 1
2 2
2 3
2 4
3 0
3 1
3 2
3 3
3 4
4 0
4 1
4 2
4 3
4 4
